**Importing Dependencies**

In [80]:
import numpy as np
import pandas as pd
import difflib         #used to get closest match of movie
from sklearn.feature_extraction.text import TfidfVectorizer #convert textual data in numerical value
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
movies_data = pd.read_csv("/content/movies.csv", skiprows=[956])


**Data Collection and Pre-processing**

In [82]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [83]:
movies_data.shape

(955, 24)

In [84]:
# selecting the revalent features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [85]:
movies_data.isnull().sum()

index                     0
budget                    0
genres                    0
homepage                445
id                        0
keywords                 10
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
production_countries      0
release_date              0
revenue                   0
runtime                   0
spoken_languages          0
status                    0
tagline                  32
title                     0
vote_average              0
vote_count                0
cast                      0
crew                      0
director                  0
dtype: int64

In [86]:
#replacing the null values as null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')


In [87]:
# combining all the fill selected feautre
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [88]:
combined_features.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
dtype: object

In [89]:
combined_features.shape

(955,)

In [90]:
#Converting the text data to numerical data
vectorizer = TfidfVectorizer()

In [91]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [92]:
print(feature_vectors)

  (0, 820)	0.16647382383550738
  (0, 2724)	0.1205549242413499
  (0, 4523)	0.20732024898792706
  (0, 3543)	0.16647382383550738
  (0, 3045)	0.2424810031108782
  (0, 5078)	0.17631183225987734
  (0, 5768)	0.21763307212610192
  (0, 4870)	0.22416893288888162
  (0, 4623)	0.21763307212610192
  (0, 5993)	0.21763307212610192
  (0, 5913)	0.21763307212610192
  (0, 4635)	0.15545111637525358
  (0, 3988)	0.2424810031108782
  (0, 3869)	0.10083687041163074
  (0, 5909)	0.12015880985950167
  (0, 5314)	0.06997388182140046
  (0, 1689)	0.25701611095747967
  (0, 4951)	0.21210708058919656
  (0, 1074)	0.22416893288888162
  (0, 5730)	0.12709078500412965
  (0, 4990)	0.3088239430716445
  (0, 2041)	0.15993796307272765
  (0, 1021)	0.23216817997270336
  (0, 1244)	0.2424810031108782
  (0, 1865)	0.09193003088117851
  :	:
  (953, 4506)	0.2477274598470698
  (953, 3774)	0.1383441075781471
  (953, 5328)	0.14906174455273263
  (953, 1803)	0.09122852323894405
  (954, 1361)	0.2829162512611576
  (954, 1474)	0.2555643336727233


Cosine Similarity

In [93]:
#getting similarity scores for using cosine similarity
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06287522 0.03485559 ... 0.00427107 0.0083541  0.        ]
 [0.06287522 1.         0.02719256 ... 0.00377853 0.00739071 0.        ]
 [0.03485559 0.02719256 1.         ... 0.0179011  0.         0.        ]
 ...
 [0.00427107 0.00377853 0.0179011  ... 1.         0.01953351 0.0556443 ]
 [0.0083541  0.00739071 0.         ... 0.01953351 1.         0.06780851]
 [0.         0.         0.         ... 0.0556443  0.06780851 1.        ]]


In [94]:
print(similarity.shape)

(955, 955)


In [95]:
#getting the movie name by user

movie_name = input("Enter the favourite movie name : ")

Enter the favourite movie name : haary


In [96]:
# creating the list with all the movie name given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [97]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)

In [98]:
print(find_close_match)

[]


In [99]:
close_match = find_close_match[0]
print(close_match)


IndexError: list index out of range

In [ ]:
#find index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

In [ ]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [ ]:
print(similarity_score)

In [ ]:
len(similarity_score)

In [ ]:
#sorting the movies based on their similarity score
sorted_similarity_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similarity_movies)

In [ ]:
# print the name of similar movie based on index
print('movies suggested for you : \n')
i=1
for movies in sorted_similarity_movies:
  index = movies[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i,'.',title_from_index)
    i+=1